# Waterbodies

* **Products used:** [DE Africa Waterbodies](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html)

## Background
The Digital Earth Africa continental Waterbodies Monitoring Service identifies more than 600,000 water bodies from over three decades of satellite observations. This service maps persistent and seasonal water bodies and the change in their water surface area over time. Mapped water bodies may include, but are not limited to, lakes, ponds, man-made reservoirs, wetlands, and segments of some river systems.

On a local, regional, and continental scale, this service helps improve our understanding of surface water dynamics and water availability and can be used for monitoring water bodies such as wetlands, lakes and dams in remote and/or inaccessible locations.

The service offers two products

* The DE Africa Waterbodies Historical Extent
* The DE Africa Waterbodies Surface Area Change

The DE Africa Waterbodies Historical Extent product is a static polygon-based view of the DE Africa Water Observations from Space All-Time Summary product. The historical extent represents where water has appeared in at least 5% of clear observations since 1987. It is not a capture of a water body’s true extent in any given year, but a record of where water has appeared since 1987. To be captured in the dataset, polygons must have an area of at least 4,500 metres squared (5 Landsat pixels). The historical extent polygons have been derived from the [DE Africa Water Observations from Space All-Time Summary](https://docs.digitalearthafrica.org/en/latest/data_specs/Landsat_WOfS_specs.html#WOfS-All-Time-Summary) product, which provides historical surface water observations across the African continent.

For each water body, the Surface Area Change product provides the percentage of the historical extent that was classified as wet, dry, or invalid, for each [DE Africa Water Observation from Space Feature Layer](https://docs.digitalearthafrica.org/en/latest/data_specs/Landsat_WOfS_specs.html#WOfS-Feature-Layer) scene that captured the water body. This can be used to identify when water bodies are increasing or decreasing in wet surface area.

**Disclaimer: DE Africa Waterbodies Surface Area Change measures the wet surface area of waterbodies as estimated from satellites. This product does not measure depth, volume, purpose of the waterbody, nor the source of the water.**

### Applications 

* Identify and analyse persistent and seasonal water bodies 
* Water resource management 
* Gain insights into the severity and spatial distribution of drought

## Description
This notebook will demonstrate how to load historical extents and wet surface area timeseries from the [DE Africa Waterbodies service](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html). 

Topics covered include:

1. Getting a list of available polygons in a given area
2. Select and plot a single polygon
3. Plotting the surface area of the polygon over time

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the “Load packages” cell.

### Load packages
Import Python packages that are used for the analysis.

In [ ]:
# Run this step while testing.
# !python -m pip install --extra-index-url="https://packages.dea.ga.gov.au" -e ../Tools/

In [ ]:
import matplotlib.pyplot as plt

from deafrica_tools.plotting import display_map
from deafrica_tools.waterbodies import (
    get_geohashes,
    get_waterbodies,
    get_waterbody,
    get_time_series,
)

## Analysis parameters

This section defines the analysis parameters, including:

* `lat`, `lon`, `buffer`: center lat/lon and analysis window size for the area of interest (in degrees)

The default area is Lac de Guiers in Senegal.

In [ ]:
# Set the central latitude and longitude
lat = 16.243016
lon = -15.842423

# Set the buffer to load around the central coordinates
buffer = 0.2

# Compute the bounding box coordinates
xlim = (lon-buffer, lon+buffer)
ylim =  (lat+buffer, lat-buffer)

# Preview area on a map
display_map(xlim, ylim)

## Getting data

The `deafrica_waterbodies` module allows you to query water bodies by location or geohash.

### List waterbody polygons in an area
We can get a list of waterbody polygons inside a bounding box of coordinates using `get_waterbodies`.

In [ ]:
# Create a bounding box from study area coordinates
bbox = (xlim[0], ylim[1], xlim[1], ylim[0])

# Select all water bodies located within the bounding box
polygons = get_waterbodies(bbox, crs="EPSG:4326")

In [ ]:
# Return GeoDataFrame with selected water bodies
polygons

The returned geodataframe includes all the water bodies which are located within the bounding box. This dataset contains metadata for each water body in the dataset, including the ID, UID, WB_UID, area, perimeter and time series. See the [Waterbodies Historical Extent documentation](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html#Waterbodies-Historical-Extent) for descriptions of each attribute.

### Displaying the polygons

Once the water body polygons are in memory, you can plot them directly, or explore them in an interactive window.

In [ ]:
# Plot the waterbody polygons located within the bounding box
polygons.plot()

In [ ]:
# Explore the waterbody polygons located within the bounding box
polygons.explore()

#### Geohashes
Every water body in DE Africa Water Bodies has a unique identifier (UID) called a [_geohash_](https://en.wikipedia.org/wiki/Geohash). The geohash of a water body is derived from its position, and this process can be reversed to obtain the location from the geohash. A waterbody's geohash is contained under the UID attribute and can be obtained through [DE Africa Maps](https://maps.digitalearth.africa/) by clicking on a waterbody.

**Note:** You might notice that these polygons have a range of similar geohash prefixes ('edyg' or 'edz'). If two geohashes have a similar prefix, this means that they are close together.

### Getting data for a specific water body

We can use the `.explore()` function above to interactively explore the subset of water bodies located within the bounding box. Hovering over a water body will display its attributes, including the water body's geohash (UID). After noting the geohash from the `.explore()` function, we can use it to extract just that water body through the `get_waterbody()` function.

The geohash for Lac de Guiers is `eduspnt9h`.

In [ ]:
selected_waterbody_geohash = "eduspnt9h"

selected_waterbody = get_waterbody(selected_waterbody_geohash)
selected_waterbody

#### Plot the selected water body

In [ ]:
selected_waterbody.plot()

#### Get the wet surface area time series for the selected waterbody

For any given geohash or a polygon, we can also use the `get_time_series()` function to get various measures of the water body surface over time. See the [Waterbodies Historical Extent documentation](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html#Waterbodies-Surface-Area-Change) for descriptions of the different surface measures.

In [ ]:
# Get time series for the selected water body
selected_waterbody_timeseries = get_time_series(waterbody=selected_waterbody.iloc[0])

# Select only the wet percentage surface measure
wet_percentage_timeseries = selected_waterbody_timeseries["pc_wet"]

# Plot the wet percentage time series and customise the y-axis label
wet_percentage_timeseries.plot()
plt.ylabel("Number of wet pixels");